<a href="https://colab.research.google.com/github/varshachawan/NaturalLanguageProcessing/blob/master/SentimentClassifier_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
import pickle

warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [10]:
df.shape

(6920, 2)

In [11]:
df.describe

<bound method NDFrame.describe of                                                       0  1
0     a stirring , funny and finally transporting re...  1
1     apparently reassembled from the cutting room f...  0
2     they presume their audience wo n't sit still f...  0
3     this is a visually stunning rumination on love...  1
4     jonathan parker 's bartleby should have been t...  1
...                                                 ... ..
6915  painful , horrifying and oppressively tragic ,...  1
6916  take care is nicely performed by a quintet of ...  0
6917  the script covers huge , heavy topics in a bla...  0
6918  a seriously bad film with seriously warped log...  0
6919  a deliciously nonsensical comedy about a city ...  1

[6920 rows x 2 columns]>

In [17]:
df[1].value_counts()

1    3610
0    3310
Name: 1, dtype: int64

In [0]:
batch_1 = df[:2000]

In [14]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [0]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


In [19]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [0]:
tokenized = batch_1[0].apply(lambda x : tokenizer.encode(x, add_special_tokens=True))

In [39]:
tokenized

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object

In [40]:
np.array(tokenized).shape

(2000,)

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [44]:
np.array(padded).shape







(2000, 59)

In [45]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [49]:
features.shape

(2000, 768)

In [0]:
labels = batch_1[1]

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [55]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
lr_clf.score(test_features, test_labels)

0.826

In [0]:
s = pickle.dumps(lr_clf)

In [61]:
from joblib import dump, load
dump(lr_clf, 'SavedModel.joblib')

['SavedModel.joblib']